In [13]:
import pandas as pd 
import json

In [120]:
train_queries = json.load(open('../data/v2_queries/query_train_prompt1_v2.json'))
test_queries = json.load(open('../data/v2_queries/query_test_prompt1_v2.json'))
train_query_df = pd.DataFrame(train_queries)
train_query_df['split'] = 'train'
test_query_df = pd.DataFrame(test_queries)
test_query_df['split'] = 'test'
full_query_df = pd.concat([train_query_df, test_query_df])

In [121]:
test_narr = json.load(open('../data/v2_narr_parsed/v2_test_set_narr.json'))
train_narr = json.load(open('../data/v2_narr_parsed/v2_train_set_narr.json'))

In [122]:
narr_df = pd.concat([
    pd.DataFrame(train_narr),
    pd.DataFrame(test_narr)
])
source_narr_df = narr_df.explode('sources').dropna().reset_index(drop=True)
source_narr_df = pd.concat([
    source_narr_df[['url']],
    source_narr_df['sources'].pipe(lambda s: pd.DataFrame(s.tolist()))
], axis=1)

In [123]:
source_df_with_clusters.head(2)

,url,Name,Original Name,Narrative Function,cluster_descript_leaf_1,cluster_descript_leaf_2,cluster_descript_leaf_3,cluster_descript_leaf_4,cluster_descript_leaf_5
0,www.theatlantic.com/sponsored/qualcomm-2016/th...,Qualcomm,Qualcomm,"""Sponsor"": This source is the sponsor of the c...","""Commercial Content"": This source is used for ...","""Promotional Materials"": These sources are use...","""Informative Source"": These sources provide co...","""Insightful Narratives"": These sources offer d...","""Credible Narratives"": These sources offer com..."
1,www.theatlantic.com/sponsored/qualcomm-2016/th...,Researchers,researchers,"""Authority"": This source is used to provide in...","""Expert Endorsement"": These sources are used t...","""Expert Authority"": These sources provide auth...","""Credible Authority"": These sources provide re...","""Expert Validation"": These sources provide rel...","""Credible Narratives"": These sources offer com..."


In [124]:
# source_narr_df[['url', 'Name', 'Perspective', 'Centrality']].merge(source_df_with_clusters, on=['url', 'Name'])

In [125]:
source_df_with_clusters = pd.read_csv('cache/2024-09-19__source-df-with-all-clusters.csv', index_col=0)

In [126]:
source_df_with_clusters_and_queries = source_df_with_clusters.merge(full_query_df)

In [129]:
to_process = ['Narrative Function', 'cluster_descript_leaf_1', 'cluster_descript_leaf_2', 'cluster_descript_leaf_3', 'cluster_descript_leaf_4', 'cluster_descript_leaf_5']
source_df_with_clusters_and_queries[to_process] = (
    source_df_with_clusters_and_queries
     [to_process]
     .apply(lambda s: s.str.split(':').str.get(0).str.split('\n\n').str.get(0).str.strip().str.replace('"', ''))
)

In [130]:
broad_clusters_categories = (
    source_df_with_clusters_and_queries
         .groupby('url')['cluster_descript_leaf_5']
         .value_counts()
         .unstack()
)

In [175]:
training_data = (
    broad_clusters_categories
        .fillna(0)
        .pipe(lambda df: (df > 0).astype(float))
        .merge(full_query_df, right_on='url', left_index=True, how='left')
)
#.pipe(lambda df: df.divide(df.sum(axis=1), axis=0))#.sort_values('Contextual Narratives', ascending=False))

# Train BERT Classifier

In [182]:
training_data.to_csv('../narrative_function/bert_training_data.csv')